In [1]:
import pandas as pd

df = pd.read_csv('test_manual.csv')
df_odorify = pd.read_excel('odorify_dataset.xlsx', sheet_name='Sheet1')

In [2]:
from rdkit import Chem

# Função para calcular InChIKey a partir de SMILES
def smiles_to_inchikey(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            return Chem.MolToInchiKey(mol)
        else:
            return None
    except:
        return None

# Calcular InChIKey para df (test_manual.csv)
df['InChIKey'] = df['SMILES'].apply(smiles_to_inchikey)

# Calcular InChIKey para df_odorify (odorify_dataset.xlsx)
# Primeiro, vamos verificar qual coluna contém os SMILES
print("Colunas do df_odorify:", df_odorify.columns.tolist())
print("\nPrimeiras linhas do df_odorify:")
print(df_odorify.head())

Colunas do df_odorify: ['Sr_No', 'IUPAC Name', 'SMILES', 'CAS', 'Odor_Status', 'Source']

Primeiras linhas do df_odorify:
   Sr_No                   IUPAC Name                 SMILES       CAS  \
0      1  1-(4-methoxyphenyl)ethanone  COC1=CC=C(C=C1)C(C)=O  100-06-1   
1      2        4-methoxybenzoic acid  COC1=CC=C(C(=O)O)C=C1  100-09-4   
2      3               ethenylbenzene       C(=C)C1=CC=CC=C1  100-42-5   
3      4               phenylmethanol        C1(=CC=CC=C1)CO  100-51-6   
4      5                 benzaldehyde       C(C1=CC=CC=C1)=O  100-52-7   

   Odor_Status      Source  
0            1  GoodScents  
1            1  GoodScents  
2            1  GoodScents  
3            1  GoodScents  
4            1  GoodScents  


In [3]:
# Calcular InChIKey para df_odorify
df_odorify['InChIKey'] = df_odorify['SMILES'].apply(smiles_to_inchikey)

# Verificar os resultados
print("df (test_manual.csv):")
print(f"Total de moléculas: {len(df)}")
print(f"InChIKeys calculados com sucesso: {df['InChIKey'].notna().sum()}")
print(f"InChIKeys faltando (erro no cálculo): {df['InChIKey'].isna().sum()}")


print("\n" + "="*80)
print("\ndf_odorify (odorify_dataset.xlsx):")
print(f"Total de moléculas: {len(df_odorify)}")
print(f"InChIKeys calculados com sucesso: {df_odorify['InChIKey'].notna().sum()}")
print(f"InChIKeys faltando (erro no cálculo): {df_odorify['InChIKey'].isna().sum()}")


[15:12:36] Explicit valence for atom # 0 N, 6, is greater than permitted


df (test_manual.csv):
Total de moléculas: 852
InChIKeys calculados com sucesso: 852
InChIKeys faltando (erro no cálculo): 0


df_odorify (odorify_dataset.xlsx):
Total de moléculas: 5860
InChIKeys calculados com sucesso: 5855
InChIKeys faltando (erro no cálculo): 5


[15:12:37] Explicit valence for atom # 0 Cl, 7, is greater than permitted
[15:12:37] Explicit valence for atom # 1 Br, 5, is greater than permitted
[15:12:37] Explicit valence for atom # 1 Cl, 3, is greater than permitted
[15:12:37] WARNING: not removing hydrogen atom without neighbors
[15:12:37] Explicit valence for atom # 0 Cl, 4, is greater than permitted
[15:12:37] WARNING: not removing hydrogen atom without neighbors


In [4]:
def comparar_dataframes(df_test, df_reference):
    """
    Compara dois dataframes e identifica moléculas únicas no df_test 
    que não estão presentes no df_reference usando InChIKey.
    
    Args:
        df_test: DataFrame de teste (test_manual.csv)
        df_reference: DataFrame de referência (odorify_dataset.xlsx)
    
    Returns:
        DataFrame contendo apenas as moléculas únicas do df_test
    """
    # Remover moléculas com InChIKey nulo (se houver)
    df_test_valido = df_test[df_test['InChIKey'].notna()].copy()
    df_reference_valido = df_reference[df_reference['InChIKey'].notna()].copy()
    
    # Identificar InChIKeys únicos (presentes em df_test mas não em df_reference)
    inchikeys_reference = set(df_reference_valido['InChIKey'])
    
    # Filtrar moléculas únicas
    df_unicas = df_test_valido[~df_test_valido['InChIKey'].isin(inchikeys_reference)].copy()
    
    # Gerar relatório
    print("="*80)
    print("RELATÓRIO DE COMPARAÇÃO DE MOLÉCULAS")
    print("="*80)
    print()
    print(f"📊 ESTATÍSTICAS GERAIS:")
    print(f"  • Total de moléculas em test_manual.csv: {len(df_test)}")
    print(f"  • Moléculas válidas em test_manual.csv: {len(df_test_valido)}")
    print(f"  • Total de moléculas em odorify_dataset.xlsx: {len(df_reference)}")
    print(f"  • Moléculas válidas em odorify_dataset.xlsx: {len(df_reference_valido)}")
    print()
    print(f"🔍 RESULTADO DA COMPARAÇÃO:")
    print(f"  • Moléculas ÚNICAS em test_manual.csv: {len(df_unicas)}")
    print(f"  • Moléculas COMUNS (presentes em ambos): {len(df_test_valido) - len(df_unicas)}")
    print()
    print(f"📈 PERCENTUAIS:")
    print(f"  • Percentual de moléculas únicas: {(len(df_unicas)/len(df_test_valido)*100):.2f}%")
    print(f"  • Percentual de moléculas comuns: {((len(df_test_valido) - len(df_unicas))/len(df_test_valido)*100):.2f}%")
    print()
    print("="*80)
    
    if len(df_unicas) > 0:
        print(f"\n📋 PRIMEIRAS 10 MOLÉCULAS ÚNICAS:")
        print(df_unicas[['SMILES', 'InChIKey', 'Outcome']].head(10).to_string(index=True))
    
    return df_unicas

# Executar a comparação
df_moleculas_unicas = comparar_dataframes(df, df_odorify)

RELATÓRIO DE COMPARAÇÃO DE MOLÉCULAS

📊 ESTATÍSTICAS GERAIS:
  • Total de moléculas em test_manual.csv: 852
  • Moléculas válidas em test_manual.csv: 852
  • Total de moléculas em odorify_dataset.xlsx: 5860
  • Moléculas válidas em odorify_dataset.xlsx: 5855

🔍 RESULTADO DA COMPARAÇÃO:
  • Moléculas ÚNICAS em test_manual.csv: 206
  • Moléculas COMUNS (presentes em ambos): 646

📈 PERCENTUAIS:
  • Percentual de moléculas únicas: 24.18%
  • Percentual de moléculas comuns: 75.82%


📋 PRIMEIRAS 10 MOLÉCULAS ÚNICAS:
                                     SMILES                     InChIKey  Outcome
16             CC1=CCC2C3C(C(C)C)CCC2(C)C13  VLXDPFLIRFYIME-UHFFFAOYSA-N        1
20     CC(=O)OCC=C(C)CCCC(C)CCCC(C)CCCC(C)C  JIGCTXHIECXYRJ-UHFFFAOYSA-N        1
26               CC1=CCC23CC1C(C)(C)C2CCC3C  IRAQOCYXUMOFCW-UHFFFAOYSA-N        1
28                           CCCC=CCCC(=O)O  PFHBCQFBHMBAMC-UHFFFAOYSA-N        1
32                    O=C([O-])C=CC(=O)[O-]  VZCYOOQTPOCHFL-UHFFFAOYSA-L  

In [5]:
# Salvar as moléculas únicas em um arquivo CSV
output_file = 'moleculas_unicas.csv'
df_moleculas_unicas.to_csv(output_file, index=False)

print(f"\n✅ Arquivo '{output_file}' salvo com sucesso!")
print(f"📁 Total de moléculas únicas salvas: {len(df_moleculas_unicas)}")
print(f"\n📄 Colunas no arquivo:")
for col in df_moleculas_unicas.columns:
    print(f"  • {col}")


✅ Arquivo 'moleculas_unicas.csv' salvo com sucesso!
📁 Total de moléculas únicas salvas: 206

📄 Colunas no arquivo:
  • SMILES
  • Outcome
  • InChIKey
